In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer

model_name = "google-bert/bert-base-uncased"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("anzeo/Slovene_SuperGLUE_BoolQ")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def preprocess_data(examples):
    passage_inputs = [f"passage : {x} " for x in examples["passage"]]
    question_inputs = [f"question : {x} " for x in examples["question"]]
    inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

    model_inputs = tokenizer(inputs, max_length=400, truncation=True)

    model_inputs["labels"] = examples["label"]
    model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import AutoModelForSequenceClassification, set_seed

set_seed(42)

id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

In [ ]:
from peft import LoHaConfig, get_peft_model, TaskType

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments

new_model_name = "loha_fine_tuned_boolq"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub(new_model_name)

In [ ]:
from transformers import pipeline

adapter_name = "anzeo/" + new_model_name

# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)